[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/weaviate/recipes/blob/main/integrations/llm-agent-frameworks/function-calling/composio/agent.ipynb)

This example demonstrates how to integrate Composio’s Gmail tool with Weaviate to create an agent that will automatically respond to Gmail messages by searching a Weaviate collection for relevant information and using that information to draft replies. A common use case for this might be to automate email support using an agent that has access to your docs/FAQs. Composio allows an AI agent/your app to easily connect to your user's apps like Gmail, Slack, Trello etc. Paired together with Weaviate to store, manage and query data, more personalized and context-aware agents can be created.

## Setup and Dependencies

In [ ]:
# Install dependencies
pip install composio-core langchain-openai langchain-weaviate
pip install langchain-community langchain pypdf python-dotenv

### Composio Setup

In the command line, run - 

In [ ]:
composio add gmail

Follow the link from your command line to set up an integration between your gmail account and Composio.

Also run -

In [ ]:
composio triggers enable gmail_new_gmail_message

This command enables a trigger for when a new mail is received in gmail. When a new email arrives on the connected account, the trigger provides data about the mail like the sender, mail content etc. 

### Set OpenAI api key

This tutorial uses an embedding model and LLM from OpenAI, for which you will need an API key set as an evironment variable.

In [ ]:
from dotenv import load_dotenv

load_dotenv()	#load environment variables

Next, we connect to a Weaviate instance. Here, we're using Weaviate Embedded. This embedded instance will stay alive for as long as the parent application is running. For a more persistent instance, you can look at the docs here.  

In [ ]:
import weaviate, os

client = weaviate.connect_to_embedded(
    headers={
        "X-OpenAI-Api-Key": os.getenv("OPENAI_API_KEY"),  # Replace with your API key
    },
)

print("Client is ready?", client.is_ready())

### Creating Weaviate Collection

Let's create a collection to store our data.

In [ ]:
from weaviate import classes as wvc

dataCollection = client.collections.create(
    name="GenerativeAI",
    vectorizer_config=wvc.config.Configure.Vectorizer.text2vec_openai(),
)

Now that we have a collection, we can import whatever data we want - 

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain_weaviate.vectorstores import WeaviateVectorStore

text_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=50)
embeddings = OpenAIEmbeddings()

loader = PyPDFLoader("Generative_artificial_intelligence.pdf", extract_images=False)
docs = loader.load_and_split(text_splitter)
WeaviateVectorStore.from_documents(docs, embeddings, client=client, index_name="GenerativeAI")

After importing the data, we can get our collection - 

In [ ]:
collection = client.collections.get("GenerativeAI")

Then, we initialize Composio's tools for Gmail - 

In [ ]:
from composio_langchain import Action, ComposioToolSet

toolset = ComposioToolSet() #Initialize Composio's Toolset

replyTool = toolset.get_actions(
    actions=[
        Action.GMAIL_REPLY_TO_THREAD, # Reply to a gmail thread
    ],
)

The `GMAIL_REPLY_TO_THREAD` action represents a function that the LLM can call. Composio provides these actions with optimized JSON schemas which makes it very easy to integrate external tools with LLMs/agentic workflows.

We are also going to define a tool that queries our Weaviate collection based on the question asked by the user -  

In [ ]:
from langchain_core.tools import StructuredTool

# Function to search Weaviate Collection

def search_collection(query: str) -> str:
    """Searches the Weaviate collection for user query and returns the results."""

    response = collection.query.hybrid(query=query, limit=3)

    stringified_response = ""
    for idx, o in enumerate(response.objects):
        stringified_response += f"Search Result: {idx+1}:\n"
        for prop in o.properties:
            stringified_response += f"{prop}:{o.properties[prop]}"
        stringified_response += "\n"
    
    return stringified_response

# Create a Structured tool from the above function

searchTool = StructuredTool.from_function(
    func=search_collection,
    name="search_collection",
    description="Searches the Weaviate collection for the given query and returns the results.",
)

tools: list[StructuredTool] = replyTool + [searchTool]

### Setting up the Agent

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_openai import ChatOpenAI

# initialize LLM
llm = ChatOpenAI(model="gpt-4o-mini")		

# The prompt can be customized to fit the needs of your app
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an AI email assistant that can write and reply to emails. You have to use the search_document tool to search the Weaviate collection for the user's query. When the user asks you a question, use the search_document tool to search for the query in the Weaviate collection and then answer the question using the search results. Send the answer back to the user in an email.",
        ),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

agent = create_tool_calling_agent(
    llm=llm,
    tools=tools,
    prompt=prompt,
)

# Create an instance of AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

Next, we create a listener for the trigger we created above - 

In [ ]:
# Create the listener
listener = toolset.create_trigger_listener()

# Attach listener to trigger and provide callback function that is executed when the trigger receives new data
@listener.callback(filters={"trigger_name": "gmail_new_gmail_message"})
def on_new_gmail_message(event) -> None:
    try:
        print("data received - ", event)

				# Extract the relevant information from the event
        headers = event.originalPayload["payload"]["headers"],
        sender = headers[16]["value"],
        query = event.originalPayload["snippet"],
        thread_id = event.originalPayload["threadId"],

        res = agent_executor.invoke({"input": f"This is the query you have to respond to: {query}. It's from {sender} and the threadId is {thread_id}."})
        print(res)
    except Exception as e:
        print("Error:", e)

print("Listener started!")
listener.listen()

```
Listener started!

data received - appName='gmail' payload={'threadId': '1916dc9e35a3401p', 'messageId': '1916dc9e35a3401p', 'messageTimestamp': '2024-08-20T03:16:59Z', 'messageText': 'I wanted to know how generative ai works for art. Can i generate images\r\nfrom gen ai models?\r\n', ...}

> Entering new AgentExecutor chain...

Invoking: `search_document` with `{'query': 'how generative ai works for art and generating images'}`


Search Result: 1:
text:OpenAI Codex.
Producing high-quality visual art is a prominent
application of generative AI.[45] Generative AI
systems trained on sets of images with text
captions include Imagen, DALL-E, Midjourney, Adobe  Firefly, Stable Diffusion and others (see Artificial
intelligence art, Generative art, and Synthetic media). They are commonly used for text-to-image generationpage:2.0source:Generative_artificial_intelligence.pdf
Search Result: 2:
text:Théâtre D'opéra Spatial, an image
generated with Midjourney
Generative artificial intelligence
Generative artificial intelligence (generative AI, GenAI,[1]
or GAI) is artificial intelligence capable of generating text,
images, videos, or other data using generative models,[2]
often in respons e to prompts.[3][4] Generative AI models
learn the patterns and structure of their input training data andpage:0.0source:Generative_artificial_intelligence.pdf
Search Result: 3:
text:artistic works. By the early 1970s , Harold Cohen was creating and exhibiting generative AI works created
by AARON, the computer program Cohen created to generate paintings.[26]
Markov chains have long been used to model natural langua ges since their development by Russian
mathematician Andrey Markov in the early 20th century. Markov published his first paper on the topic inpage:1.0source:Generative_artificial_intelligence.pdf

Invoking: `gmail_reply_to_thread` with `{'thread_id': '1916dc9e35a3401p', 'message_body': 'Dear John,\n\nThank you for your inquiry about generative AI and its application in art. \n\nGenerative AI works by utilizing models that can generate text, images, videos, or other data based on training data. In the context of art, it involves systems trained on large datasets of images paired with text captions. Some popular generative AI models for image generation include DALL-E, Midjourney, and Stable Diffusion. These models learn the patterns and structures from the training data, allowing them to create new and unique images in response to prompts.\n\nSo yes, you can definitely generate images using generative AI models!\n\nBest regards,\n[Your Name]', ...}

[2024-08-20 08:48:08,645][INFO] Executing action: GMAIL_REPLY_TO_THREAD
{'execution_details': {'executed': True}, 'response_data': {'id': '1916dcac6c9424e9', 'threadId': '1916dc9e35a3401p', 'labelIds': ['SENT']}}I have replied to John's inquiry about how generative AI works for art and the possibility of generating images using generative AI models. If you need any further assistance, feel free to ask!

> Finished chain.

### Connect with Composio and learn more

If you encounter any problems, please let us know at our [Discord](https://discord.com/invite/cNruWaAhQk).

Check out [Composio's documentation](https://docs.composio.dev/introduction/intro/overview) to learn more about how to use and integrate various tools for different usecases.